In [1]:
from qec_benchmarks import *
from archiver4qiskit2 import *
from qiskit import IBMQ
from qiskit.visualization import plot_gate_map

import matplotlib.pyplot as plt

In [4]:
def diff(p0,p1):
    return (p1-p0)/(1-2*p0)

def get_probs(bench,dd_override=None,postselected=False):
    
    min_probs = {}
    max_probs = {}
    diff_probs = {}
    for log in ['0','1','+']:
        
        if dd_override==None:
            dd = log=='+'
        else:
            dd = dd_override
           
        if postselected:
            between_probs = bench.postselected_between_probs()
        else:
            between_probs = bench.between_probs()
        
        betweens = {dtype: between_probs[dtype,dd] for dtype in ['min','max']}

        min_probs[log] = []
        max_probs[log] = []
        diff_probs[log] = []
        for q in bench.mid_qubits:
            min_probs[log].append(betweens['min'][q][log][1])
            max_probs[log].append(betweens['max'][q][log][1])
            diff_probs[log] = []
            for p0,p1 in zip(min_probs[log],max_probs[log]):
                diff_probs[log].append(diff(p0,p1))
                
    for probs in [min_probs, max_probs]:
        probs['01'] = []
        for p01,p10 in zip(probs['0'],probs['1']):
            probs['01'].append( (p01+p10)/2 )
            
    return min_probs, max_probs, diff_probs

def plot_probs(bench, probs, title):

    plt.figure(figsize=(7.5,5))
    plt.rcParams.update({'font.size': 16})
    plt.rcParams.update({'figure.autolayout': True})
    plt.title(title)

    plt.scatter(bench.mid_qubits, probs)
    plt.savefig(title.replace(' ','_')+'.png')
    
def get_ids(results):
    minimal_ids = {}
    rep_ids = {}
    for backend_name in results:
        minimal_ids[backend_name] = results[backend_name][0]
        rep_ids[backend_name] = results[backend_name][1]
    return minimal_ids, rep_ids

def ps2hex(ps,pmax): 
    hps = []
    for p in ps:
        hp = hex(int(255*(p/pmax)))[2::]
        if len(hp)==1:
            hp = '0'+ hp
        hps.append(hp)
        
    return '#'+hps[0]+hps[1]+hps[2]

def plot_device_probs(bench, probs, logs=['01',None,'+'], dec=1):

    pmax = 0
    for log in logs:
        if log:
            pmax = max(pmax,max(probs[log]))

    color = []
    label = []
    for q in range(27):
        if q not in bench.mid_qubits:
            color.append('#dddddd')
            label.append('')
        else:
            j = bench.mid_qubits.index(q)
            hp = {}
            pc = {}
            for log in logs:
                if log:
                    p = probs[log][j]
                    hp[log] = hex(int(255*(p/pmax)))[2::]
                    if len(hp[log])==1:
                        hp[log] = '0'+hp[log]
                    pc[log] = str(int(10**dec * 100*p)/(10**dec))+'%'
                else:
                    hp[log] = '00'
                    pc[log] = '0%'

            hc = []
            for log in logs:
                hc.append(hp[log])
            color.append('#'+hc[0]+hc[1]+hc[2])      

            lab = ''
            for log in logs:
                if log:
                    lab += pc[log]
                    lab +='\n'
            lab = lab[:-1]
            label.append(lab)

    fig = plot_gate_map(bench.backend, qubit_color=color, qubit_labels=label, figsize=(12,12), line_color=['grey']*100)

    return fig

def plot_device_ts(backend, cx_pmax, date=None, dec=1):

    coupling_map = backend.configuration().coupling_map
    n = backend.configuration().n_qubits

    if date:
        properties = backend.properties(datetime=date)
    else:
        properties = backend.properties()
    
    p1 = []
    p2 = []
    for q in range(n):
        t1 = properties.t1(q)
        t2 = properties.t2(q)
        tr = properties.readout_length(q)

        p1.append( 1-np.exp(-(tr/2)/t1) )
        p2.append( (1-np.exp(-(tr)/t2))/2 )

    pmax = max(max(p1),max(p2))
    t_colors = []
    t_labels = []
    for q in range(n):
        t_colors.append(ps2hex([p1[q],0,p2[q]],pmax))

        label = ''
        for p in [p1[q],p2[q]]:
            pc = str(int(10**dec*100*p)/(10**dec))+'%'
            label += pc + '\n'
        t_labels.append(label[:-1])


    cx_errors = []
    for qubits in coupling_map:
        p = properties.gate_error('cx',qubits)
        cx_errors.append(p)

    cx_colors = []
    for p in cx_errors:
        if p:
            if p<cx_pmax:
                cx_colors.append(ps2hex([0,p,0],cx_pmax))
            else:
                cx_colors.append('grey') 
        else:
            cx_colors.append('grey')

    line_color=['grey']*100
    line_color[3] = 'red'

    return plot_gate_map(bench.backend, qubit_color=t_colors, qubit_labels=t_labels, line_color=cx_colors, line_width=10, figsize=(12,12))


In [5]:
# from mystery.ipynb
# (..., '[None, None]', '[2, 0]')
default_full = {
    'ibm_cairo':
        [
            '625558eb8233e8188cad1aa4@ibm_cairo@ibm-q-afrl_sleeq_main',
            {
                ('{1: [912, 912], 2: [880, 880], 4: [592, 592], 3: [1040, 1040], 5: [400, 400], 8: [1072, 1072], 11: [976, 976], 12: [976, 976], 14: [1040, 1040], 13: [528, 528], 15: [1040, 1040], 16: [1104, 1104], 18: [400, 400], 19: [1040, 1040], 21: [1040, 1040], 22: [976, 976], 23: [1040, 1040], 25: [1040, 1040], 24: [976, 976]}',True): '625558f9d8e31ab9a78b4cc9@ibm_cairo@ibm-q-afrl_sleeq_main',
                ('{1: [912, 912], 2: [880, 880], 4: [592, 592], 3: [1040, 1040], 5: [400, 400], 8: [1072, 1072], 11: [976, 976], 12: [976, 976], 14: [1040, 1040], 13: [528, 528], 15: [1040, 1040], 16: [1104, 1104], 18: [400, 400], 19: [1040, 1040], 21: [1040, 1040], 22: [976, 976], 23: [1040, 1040], 25: [1040, 1040], 24: [976, 976]}',False): '625558fd42d2a41f5d7d4132@ibm_cairo@ibm-q-afrl_sleeq_main',
                ('{1: [40336, 55792], 2: [51552, 42416], 4: [38704, 11648], 3: [50544, 65280], 5: [47072, 29136], 8: [62608, 45872], 11: [50384, 85120], 12: [35072, 44496], 14: [65104, 113472], 13: [54976, 42672], 15: [64384, 122080], 16: [61360, 101280], 18: [57568, 39360], 19: [85536, 95744], 21: [72560, 57248], 22: [55728, 61296], 23: [22448, 42032], 25: [46256, 92304], 24: [111888, 163152]}',True): '625559013bfc7674e4eac3cf@ibm_cairo@ibm-q-afrl_sleeq_main',
                ('{1: [40336, 55792], 2: [51552, 42416], 4: [38704, 11648], 3: [50544, 65280], 5: [47072, 29136], 8: [62608, 45872], 11: [50384, 85120], 12: [35072, 44496], 14: [65104, 113472], 13: [54976, 42672], 15: [64384, 122080], 16: [61360, 101280], 18: [57568, 39360], 19: [85536, 95744], 21: [72560, 57248], 22: [55728, 61296], 23: [22448, 42032], 25: [46256, 92304], 24: [111888, 163152]}',False): '625559078233e8656fad1aa5@ibm_cairo@ibm-q-afrl_sleeq_main'
            }
        ],
    
    'ibm_hanoi':
        [
            '625558b38b86e90cfac0d1bd@ibm_hanoi@ibm-q-afrl_sleeq_main',
            {
                ('{1: [1248, 1248], 2: [160, 160], 4: [1216, 1216], 7: [1216, 1216], 10: [128, 128], 11: [1216, 1216], 12: [1216, 1216], 14: [1216, 1216], 13: [1216, 1216], 15: [1216, 1216], 16: [1216, 1216], 18: [1216, 1216], 19: [1216, 1216], 21: [1216, 1216], 22: [1216, 1216], 23: [1216, 1216], 25: [1216, 1216], 24: [1216, 1216]}',True): '625558bd3bfc767795eac3ce@ibm_hanoi@ibm-q-afrl_sleeq_main',
                ('{1: [1248, 1248], 2: [160, 160], 4: [1216, 1216], 7: [1216, 1216], 10: [128, 128], 11: [1216, 1216], 12: [1216, 1216], 14: [1216, 1216], 13: [1216, 1216], 15: [1216, 1216], 16: [1216, 1216], 18: [1216, 1216], 19: [1216, 1216], 21: [1216, 1216], 22: [1216, 1216], 23: [1216, 1216], 25: [1216, 1216], 24: [1216, 1216]}',False): '625558c3d8e31a673d8b4cc8@ibm_hanoi@ibm-q-afrl_sleeq_main',
                ('{1: [136880, 61376], 2: [86320, 123424], 4: [85072, 15840], 7: [166736, 149392], 10: [86496, 116768], 11: [98064, 136816], 12: [126304, 116976], 14: [118048, 15296], 13: [163952, 145952], 15: [98672, 24384], 16: [127392, 170016], 18: [76800, 136528], 19: [102336, 144240], 21: [75728, 20992], 22: [79968, 54784], 23: [106064, 26880], 25: [135600, 63568], 24: [60640, 24304]}',True): '625558c98233e8830dad1aa3@ibm_hanoi@ibm-q-afrl_sleeq_main',
                ('{1: [136880, 61376], 2: [86320, 123424], 4: [85072, 15840], 7: [166736, 149392], 10: [86496, 116768], 11: [98064, 136816], 12: [126304, 116976], 14: [118048, 15296], 13: [163952, 145952], 15: [98672, 24384], 16: [127392, 170016], 18: [76800, 136528], 19: [102336, 144240], 21: [75728, 20992], 22: [79968, 54784], 23: [106064, 26880], 25: [135600, 63568], 24: [60640, 24304]}',False): '625558cf42d2a474597d4130@ibm_hanoi@ibm-q-afrl_sleeq_main'
            }
        ]   
}

# from mystery-3.ipynb
# (..., '[[2, 3, 4], None]', '[2, 0]')
default_targeted = {
    'ibmq_montreal':[None,{('{1: [960, 960], 2: [6240, 6240], 4: [6240, 6240], 3: [6240, 6240], 5: [6240, 6240], 7: [6240, 6240], 8: [6240, 6240], 10: [6240, 6240], 11: [6240, 6240], 12: [960, 960], 14: [6240, 6240], 13: [6240, 6240], 15: [6240, 6240], 16: [6240, 6240], 18: [6240, 6240], 19: [6240, 6240], 21: [6240, 6240], 22: [6240, 6240], 23: [6240, 6240], 25: [160, 160], 24: [6240, 6240]}', True): '6269459d63109304138f55fb@ibmq_montreal@ibm-q-afrl_sleeq_main',('{1: [102960, 25840], 2: [69536, 62176], 4: [89488, 122016], 3: [55888, 24624], 5: [57024, 56224], 7: [81296, 61408], 8: [55952, 68336], 10: [77424, 92000], 11: [74576, 33216], 12: [69424, 82688], 14: [61680, 103920], 13: [83328, 37216], 15: [66464, 69424], 16: [65536, 38576], 18: [46320, 9264], 19: [81584, 71296], 21: [27664, 35440], 22: [85184, 52880], 23: [75936, 42752], 25: [64640, 55248], 24: [58624, 18784]}', True): '626945a662832d28b53e7d8c@ibmq_montreal@ibm-q-afrl_sleeq_main'}],
    'ibmq_toronto':[None,{('{4: [736, 736], 7: [144, 144], 8: [144, 144], 10: [144, 144], 11: [144, 144], 12: [144, 144], 14: [6224, 6224], 13: [6224, 6224], 15: [6224, 6224], 16: [144, 144], 18: [6224, 6224], 19: [144, 144], 21: [6224, 6224], 22: [144, 144], 23: [144, 144], 25: [144, 144], 24: [144, 144]}', True): '626945af48ee1c7ab8542a45@ibmq_toronto@ibm-q-afrl_sleeq_main',('{4: [83920, 70720], 7: [63488, 164896], 8: [74112, 112096], 10: [68608, 110368], 11: [46496, 61824], 12: [65184, 64048], 14: [75088, 65824], 13: [79056, 92352], 15: [33168, 33856], 16: [65792, 79024], 18: [83312, 87216], 19: [71072, 76064], 21: [38112, 28832], 22: [32960, 68992], 23: [37152, 25856], 25: [69888, 71232], 24: [107968, 88688]}', True): '626945bf63109333248f55fd@ibmq_toronto@ibm-q-afrl_sleeq_main'}],
    'ibmq_mumbai':[None,{('{1: [544, 544], 2: [544, 544], 4: [544, 544], 3: [544, 544], 7: [544, 544], 10: [544, 544], 12: [544, 544], 14: [544, 544], 13: [544, 544], 15: [544, 544], 16: [544, 544], 18: [544, 544], 19: [544, 544], 21: [544, 544], 22: [544, 544], 23: [544, 544], 25: [544, 544], 24: [544, 544]}', True): '626945c88525d1532a8f8242@ibmq_mumbai@ibm-q-afrl_sleeq_main', ('{1: [60624, 88384], 2: [110496, 57040], 4: [83616, 24400], 3: [82816, 130240], 7: [131456, 58288], 10: [78544, 97056], 12: [117968, 197216], 14: [97296, 130064], 13: [98144, 19392], 15: [45600, 126736], 16: [72688, 60960], 18: [143872, 167920], 19: [124272, 117984], 21: [74480, 85168], 22: [69696, 45232], 23: [51888, 17376], 25: [110000, 42160], 24: [165616, 49568]}', True): '626945cf179c85fdb6ffb29a@ibmq_mumbai@ibm-q-afrl_sleeq_main'}],
    'ibm_hanoi':[None,{('{1: [1248, 1248], 2: [160, 160], 4: [1216, 1216], 7: [1216, 1216], 10: [128, 128], 11: [1216, 1216], 12: [1216, 1216], 14: [1216, 1216], 13: [1216, 1216], 15: [1216, 1216], 16: [1216, 1216], 18: [1216, 1216], 19: [1216, 1216], 21: [1216, 1216], 22: [1216, 1216], 23: [1216, 1216], 25: [1216, 1216], 24: [1216, 1216]}', True): '626945d9179c85c8abffb29b@ibm_hanoi@ibm-q-afrl_sleeq_main', ('{1: [81392, 85456], 2: [49776, 73968], 4: [97280, 15856], 7: [90368, 133872], 10: [88608, 184176], 11: [124928, 99504], 12: [102832, 107120], 14: [79984, 15296], 13: [96624, 92880], 15: [119104, 25888], 16: [123664, 128048], 18: [92528, 83664], 19: [77776, 94784], 21: [76976, 20816], 22: [86672, 46992], 23: [105024, 30368], 25: [66688, 60304], 24: [99072, 23184]}', True): '626945e28525d19be58f8243@ibm_hanoi@ibm-q-afrl_sleeq_main'}],
    'ibm_cairo':[None,{('{1: [912, 912], 2: [880, 880], 4: [592, 592], 3: [1040, 1040], 5: [400, 400], 8: [1072, 1072], 11: [976, 976], 12: [976, 976], 14: [1040, 1040], 13: [528, 528], 15: [1040, 1040], 16: [1104, 1104], 18: [400, 400], 19: [1040, 1040], 21: [1040, 1040], 22: [976, 976], 23: [1040, 1040], 25: [1040, 1040], 24: [976, 976]}', True): '626945f063109364238f55fe@ibm_cairo@ibm-q-afrl_sleeq_main', ('{1: [2752, 4496], 2: [50784, 40848], 4: [34832, 13888], 3: [92016, 109712], 5: [57520, 37312], 8: [54784, 44128], 11: [41792, 70400], 12: [63936, 109904], 14: [45920, 62160], 13: [62656, 99936], 15: [87520, 158560], 16: [58880, 107968], 18: [58704, 68048], 19: [70304, 119520], 21: [60752, 62784], 22: [55120, 67712], 23: [53056, 82048], 25: [54384, 55440], 24: [60416, 106176]}', True): '626945f8f01f4d4bb9ea4521@ibm_cairo@ibm-q-afrl_sleeq_main'}],
    'ibm_auckland':[None,{('{1: [720, 720], 2: [720, 720], 4: [720, 720], 3: [960, 960], 5: [720, 720], 7: [720, 720], 8: [1344, 1344], 10: [960, 960], 11: [1344, 1344], 12: [720, 720], 14: [1344, 1344], 13: [720, 720], 15: [720, 720], 16: [720, 720], 18: [720, 720], 19: [720, 720], 21: [720, 720], 22: [720, 720], 23: [720, 720], 25: [720, 720], 24: [720, 720]}', True): '626946035efe734f59505a6f@ibm_auckland@ibm-q-afrl_sleeq_main', ('{1: [122864, 158992], 2: [63312, 94096], 4: [135744, 142240], 3: [115136, 39680], 5: [116528, 43088], 7: [90432, 146208], 8: [95792, 59008], 10: [118656, 29232], 11: [79296, 60576], 12: [112720, 138112], 14: [56720, 75856], 13: [104320, 14144], 15: [66960, 45072], 16: [112192, 90816], 18: [111808, 108464], 19: [106176, 14640], 21: [73552, 79040], 22: [70400, 41536], 23: [117520, 34480], 25: [95328, 121600], 24: [79568, 10288]}', True): '6269460a5efe737b10505a70@ibm_auckland@ibm-q-afrl_sleeq_main'}],
}
#    'ibm_peekskill':[None,{('{1: [400, 400], 2: [480, 480], 4: [416, 416], 3: [400, 400], 5: [416, 416], 7: [544, 544], 8: [400, 400], 10: [480, 480], 11: [416, 416], 12: [624, 624], 14: [144, 144], 13: [416, 416], 15: [144, 144], 16: [336, 336], 19: [400, 400], 22: [416, 416], 25: [400, 400]}', True): '62ac69e4f09ece43d3e388af@ibm_peekskill@ibm-q-afrl_sleeq_main', ('{1: [400, 400], 2: [480, 480], 4: [416, 416], 3: [400, 400], 5: [416, 416], 7: [544, 544], 8: [400, 400], 10: [480, 480], 11: [416, 416], 12: [624, 624], 14: [144, 144], 13: [416, 416], 15: [144, 144], 16: [336, 336], 19: [400, 400], 22: [416, 416], 25: [400, 400]}', False): '62ac69ea5e6b2ada2481d9e9@ibm_peekskill@ibm-q-afrl_sleeq_main', ('{1: [282176, 61136], 2: [271616, 136752], 4: [245392, 135792], 3: [164320, 151248], 5: [250368, 204672], 7: [229104, 285776], 8: [145472, 248352], 10: [180352, 68128], 11: [23872, 30544], 12: [188000, 273136], 14: [204592, 101968], 13: [37984, 22320], 15: [183744, 118800], 16: [200384, 28608], 19: [153248, 65616], 22: [229168, 111824], 25: [146736, 244912]}', True): '62ac69ef00686255e3087f31@ibm_peekskill@ibm-q-afrl_sleeq_main', ('{1: [282176, 61136], 2: [271616, 136752], 4: [245392, 135792], 3: [164320, 151248], 5: [250368, 204672], 7: [229104, 285776], 8: [145472, 248352], 10: [180352, 68128], 11: [23872, 30544], 12: [188000, 273136], 14: [204592, 101968], 13: [37984, 22320], 15: [183744, 118800], 16: [200384, 28608], 19: [153248, 65616], 22: [229168, 111824], 25: [146736, 244912]}', False): '62ac69f3292790178fbadfc9@ibm_peekskill@ibm-q-afrl_sleeq_main'}]
#}
                     

In [7]:
backend_name = 'ibm_hanoi'


In [8]:
results = default_full

minimal_ids, default_rep_ids = get_ids(results)
bench = RepetitionBenchmark(backend_name, archive_ids=default_rep_ids[backend_name], use_old=False, run=False)


BackendPropertyError: "Couldn't find the property 'T2' for qubit 14."